In [1]:
##vidsuaslizations fine in JN
#nlp visualizations 
## we can do a heatmap of words or phrases that are "spammy"
##we can do a word cloud of words or phrases that are "spammy"
#unsupervised learning

In [1]:
#lets get crackn
import os
import chardet #for detecting encoding
from email import policy
from email.parser import BytesParser
#^^these two above work for MIME format^^
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer #vectoring is after preporcessing
from sklearn.model_selection import train_test_split
#download stuff for nltk if need be
#nltk.download('stopwords')
#^^this is to update stopwords^^

In [3]:
#helful starter code from zeb
import mimetypes
#needs corpus beforehand
class Corpus:
    def __init(self):
        self.documents = []
    def add_document(self, document):
        self.documents.append(document)

def add_mime_text_files_to_corpus(corpus, directory):
  """Adds all MIME text files in the given directory to the given corpus.

  Args:
    corpus: A corpus object.
    directory: The directory containing the MIME text files.
  """

  for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    mimetype = mimetypes.guess_type(filepath)[0]
    if mimetype == "text/plain":
      with open(filepath, "r") as f:
        corpus.add_document(f.read())

# Example usage:
corpus = Corpus()
add_mime_text_files_to_corpus(corpus, "Resources/spam_uno")

In [4]:
#helful starter code from zeb
import nltk
from nltk.corpus import PlaintextCorpusReader

# Create a corpus reader for the MIME text files
corpus_reader = PlaintextCorpusReader('Resources/spam_uno', '.*\.txt')

# Add the corpus to NLTK
#nltk.corpus.reader.add_corpus(corpus_reader)
#^^AttributeError: module 'nltk.corpus.reader' has no attribute 'add_corpus'^^

# Print the contents of the corpus
print(corpus_reader.raw())

# Get the contents of the first file in the corpus
file_contents_example = corpus_reader.raw('00284.4cdf4c9e9404c79c85ab5ac12ce39e85')

# Print the contents of the file
print(file_contents_example)

AttributeError: module 'nltk.corpus.reader' has no attribute 'add_corpus'

In [20]:
#lets build an oven (corpus)
##ham_folder_easy_uno = 'Resources/easy_ham_uno'
##spam_folder_uno = 'Resources/spam_uno'
##ham_test_folder = 'Resources/ham_testing_renaming'
#^^paths to our email datasets^^
#we need to mark which emails are spam or ham so have lists for emails and labels
emails = []
labels = []
#read the text files from our folders and give them labels
def read_files(folder, label):
    files = os.listdir(folder)
    for file in files:
        file_path = os.path.join(folder,file)
        #we ask chardet to inspect our meat
        with open(file_path,'rb') as f:
            #UTF-8 does not work, Windows-1252 works!
            ##use chardet if we have any hiccups...
            #https://stackoverflow.com/questions/436220/how-to-determine-the-encoding-of-text
            #^^windows-1252 only reads the first line...^^
            uncooked_data = f.read()
            result = chardet.detect(uncooked_data)
            encoding = result['encoding']
        #now with correct encoding, read and append
        with open(file_path, 'r', encoding=encoding) as f:
            emails.append(f.read())
            labels.append(label)
#read ham and spam
##read_files(ham_test_folder, 'test')
read_files(ham_folder_easy_uno, 'ham')
##read_files(spam_folder_uno, 'spam')

In [26]:
#ham_test_email = 'Resources/ham_testing_renaming/testingEasyHamUno00001.txt'
#lets build an oven (corpus)
ham_folder_easy_uno = 'Resources/easy_ham_uno'
#spam_folder_uno = 'Resources/spam_uno'
##ham_test_folder = 'Resources/ham_testing_renaming'
#^^paths to our email datasets^^
#we need to mark which emails are spam or ham so have lists for emails and labels
email_subjects = []
email_bodies = []
labels = []
def parse_mime_email(file_path):
    # Read the email file in binary mode
    with open(file_path, 'rb') as file:
        mime_content = file.read()
    #parsing the MIME content
    email_message = BytesParser(policy=policy.default).parsebytes(mime_content)
    #extract subject
    email_subject = email_message['subject']
    #initialize a variable to hold the email body
    email_body = ""
    #check if the email is multipart (less headaches
    #well now we need a try/except because of another hurdle
    ##try: 
    if email_message.is_multipart():
            #iterate through each part
        for part in email_message.walk():
            if part.get_content_type() == "text/plain" and not email_body:
                    #get the text content
                email_body = part.get_payload(decode=True).decode(part.get_content_charset, 'ignore')
                break
            elif part.get_content_type() == "text/html" and not email_body:
                    email_body = part.get_payload(decode=True).decode(part.get_content_charset(), 'ignore')
    else:
        email_body = email_message.get_payload(decode=True).decode(email_message.get_content_charset, 'ignore')
    ##except Exception as e:
        ##print(f"It is I sir: {file_path}: {e}")
       ## email_subject = "Error"
      ##  email_body = "This email is not helpful."
    return email_subject, email_body
#show it works
##subject, email_body = parse_mime_email(ham_test_email)
##print("Subject:", subject)
##print("Email Body:", email_body)
##works! now we need to see how to read all of them at the same time... we can use another function, maybe nest
#want to also include sender, but it may give bias as to who the email is...
#from and only look for those rather than patterns of the sender...
#read the text files from our folders and give them labels
def read_files(folder, label):
    files = os.listdir(folder)
    for file in files:
        file_path = os.path.join(folder,file)
        if os.path.isfile(file_path):
        #no encoding needed!
        ##atempt
            try:
        #we need to specify what we want using our mime parsing function
                email_subject, email_body = parse_mime_email(file_path)
                email_subjects.append(email_subject)#using email_subject variable because parse function already did it...
                email_bodies.append(email_body) #using email_body variable because parse function already did it...
                labels.append(label)
                print(email_body)
            except Exception as e:
                print(f"It is I sir: {file_path}: {e}")
    return email_subjects, email_bodies, labels
#read ham and spam
##read_files(ham_test_folder, 'test')
read_files(ham_folder_easy_uno, 'ham')
##read_files(spam_folder_uno, 'spam')
#print(email_subjects)

It is I sir: Resources/easy_ham_uno/00769.25bf9a767b5db0ed93f03c1637281663: decode() argument 'encoding' must be str, not method
It is I sir: Resources/easy_ham_uno/01074.8590d61ac0aeeadb58dc2f2ba776c406: decode() argument 'encoding' must be str, not method
It is I sir: Resources/easy_ham_uno/02479.14365bcad3a60fcf24c5c1813f6291fb: decode() argument 'encoding' must be str, not method
It is I sir: Resources/easy_ham_uno/01417.ce7b07a2114218dbac682b599785820d: decode() argument 'encoding' must be str, not method
It is I sir: Resources/easy_ham_uno/00357.d559b71616f64ba1d1c1e61a03644fd4: decode() argument 'encoding' must be str, not method
It is I sir: Resources/easy_ham_uno/01465.b1fa1feb0603eff01231fa4a5a74cf37: decode() argument 'encoding' must be str, not method
It is I sir: Resources/easy_ham_uno/01083.68ca1eec938ec95c00e6fcd3ed282dd3: decode() argument 'encoding' must be str, not method
It is I sir: Resources/easy_ham_uno/01235.ba9c966dd1894a1206da779f572da412: decode() argument 'en

([], [], [])

In [3]:
#lets see if the oven cooks everything evenly
#test if our list is properly read and we see the same amount of emails as we asked it to read
ham_test_df = pd.DataFrame({'email_subject':email_subjects, 'email_body':email_bodies, 'labels':labels})
##print(ham_test_df.head())
#save as csv
import csv
ham_test_df.to_csv('Resources/Labeled_CSVs/ham_testing_labeled.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8-sig')
#not all data is shown in csv, but good in text edit. we can try "fixing it for excel, or try json.